In [1]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup

import pandas as pd

save_recipe_fname = './data/recipe_main_with_thumbnails.csv'

# 데이터 저장 경로 확인 및 디렉토리 생성
if not os.path.exists('./data'):
    os.makedirs('./data')

by_type = {'밑반찬': '63', '메인반찬': '56', '국/탕': '54', '찌개': '55', '디저트': '60', '면/만두': '53',
           '밥/죽/떡': '52', '퓨전': '61', '양념/잼/소스': '58', '양식': '65', '샐러드': '64', '스프': '68',
           '빵': '66', '과자': '69', '차/음료/술': '59'}  # cat4
by_situation = {'일상': '12', '초스피드': '18', '손님접대': '13', '술안주': '19', '다이어트': '21',
                '도시락': '15', '영양식': '43', '간식': '17', '야식': '45', '명절': '44'}  # cat2
by_ingredient = {'소고기': '70', '돼지고기': '71', '닭고기': '72', '육류': '23', '채소류': '28', '해물류': '24',
                 '달걀/유제품': '50', '쌀': '47', '밀가루': '32', '건어물류': '25', '버섯류': '31', '과일류': '48',
                 '곡류': '26'}  # cat3
by_method = {'볶음': '6', '끓이기': '1', '부침': '7', '조림': '36', '무침': '41', '비빔': '42',
             '찜': '8', '절임': '10', '튀김': '9', '삶기': '38', '굽기': '67', '회': '37'}  # cat1

recipe_idx = 9401
list4df = []
print("시작")
try:
    for type_key, type_value in by_type.items():
        for situ_key, situ_value in by_situation.items():
            for ing_key, ing_value in by_ingredient.items():
                for method_key, method_value in by_method.items():
                    try:
                        main_url = 'https://www.10000recipe.com/recipe/list.html?q=&query=&' \
                                   'cat1={m}&cat2={s}&cat3={i}&cat4={t}' \
                                   '&fct=&order=reco&lastcate=cat4&dsearch=&copyshot=&scrap=&degree=&portion=&time=&niresource='\
                            .format(m=method_value, s=situ_value, i=ing_value, t=type_value)
                        response = requests.get(main_url, headers={'User-Agent': 'Mozilla/5.0'})
                        time.sleep(3)

                        if response.status_code == 200:  # 정상 연결시
                            soup = BeautifulSoup(response.text, 'html.parser')
                            page_len = len(soup.select('#contents_area_full > ul > nav > ul > li'))
                            for page in range(1001, min(2001, page_len + 1)):
                                if page != 1:
                                    main_url = main_url + '&page=' + str(page)
                                    response = requests.get(main_url, headers={'User-Agent': 'Mozilla/5.0'})
                                    soup = BeautifulSoup(response.text, 'html.parser')
                                    time.sleep(1)
                                sources = soup.select('#contents_area_full > ul > ul > li > div.common_sp_thumb > a')
                                for source in sources:
                                    try:
                                        recipe_url = 'https://www.10000recipe.com' + source['href']
                                        thumbnail_url = source.find('img')['src']  # 썸네일 이미지 URL 추출
                                        response_r = requests.get(recipe_url, headers={'User-Agent': 'Mozilla/5.0'})
                                        soup_r = BeautifulSoup(response_r.text, 'html.parser')
                                        time.sleep(1)

                                        # 각 데이터를 추출할 때, 해당 요소가 존재하는지 확인
                                        title = soup_r.select('#contents_area > div.view2_summary.st3 > h3')[0].text if soup_r.select('#contents_area > div.view2_summary.st3 > h3') else None
                                        
                                        # 인분, 조리시간, 난이도 추출
                                        servings = soup_r.select('#contents_area > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info1')[0].text if soup_r.select('#contents_area > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info1') else None
                                        servings = re.sub(r'[^0-9]', '', servings) if servings else None
                                        cooking_time = soup_r.select('#contents_area > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info2')[0].text if soup_r.select('#contents_area > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info2') else None
                                        difficulty = soup_r.select('#contents_area > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info3')[0].text if soup_r.select('#contents_area > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info3') else None

                                        # 재료 추출
                                        ingredient_s = soup_r.select('#divConfirmedMaterialArea > ul')
                                        ingredient = []
                                        if ingredient_s:
                                            for ing in ingredient_s[0].find_all('li'):
                                                ingredient.append(ing.text.split('\n')[0].strip())

                                        # 조리순서
                                        cooking_order = []
                                        process = 1
                                        while True:
                                            cooking_order_s = soup_r.select(f'#stepdescr{process}')
                                            if cooking_order_s:
                                                cooking_order.append(cooking_order_s[0].text.strip())
                                                process += 1
                                            else:
                                                break

                                        list4df.append([recipe_idx, type_key, situ_key, ing_key, method_key, title, recipe_url, thumbnail_url, servings, cooking_time, difficulty, ingredient, cooking_order])
                                        recipe_idx += 1
                                        if recipe_idx % 200 == 0:
                                            recipe_df = pd.DataFrame(list4df,
                                                                     columns=['index', '종류별', '상황별', '재료별', '방법별', '제목', 'url', '썸네일 URL', '인분',
                                                                              '조리시간', '난이도', '재료', '조리순서'])
                                            recipe_df.to_csv(save_recipe_fname, encoding='utf-8', mode='a', header=recipe_idx == 200, index=False)
                                            list4df.clear()  # 데이터를 저장 후 리스트를 초기화

                                            print(f"{recipe_idx}번째 레시피 저장 완료")

                                    except Exception as e:
                                        print(f"썸네일 추출 오류: {e}")
                                        continue
                                time.sleep(1)  # 다음 페이지로 넘어가기 전 대기
                    except Exception as e:
                        print(f"Error processing recipe {recipe_url}: {e}")
                        continue  # 오류가 발생하면 해당 레시피를 건너뛰고 계속 진행

except Exception as e:
    print(f"An error occurred: {e}")

# 최종적으로 DataFrame을 저장 (지정된 열만 포함)
recipe_df = pd.DataFrame(list4df, columns=['index', '종류별', '상황별', '재료별', '방법별', '제목', 'url', '썸네일 URL', '인분', '조리시간', '난이도', '재료', '조리순서'])

# 전체 열 출력 옵션 설정 (디버깅용)
pd.set_option('display.max_columns', None)

# 선택된 열 출력
print(recipe_df[['index', '종류별', '상황별', '재료별', '방법별', '제목', 'url', '썸네일 URL', '인분', '조리시간', '난이도', '재료', '조리순서']])

# CSV 파일로 저장 (지정된 열만 포함)
recipe_df.to_csv(save_recipe_fname, encoding='utf-8', mode='a', header=recipe_idx == 200, index=False)


시작
An error occurred: name 'recipe_url' is not defined
Empty DataFrame
Columns: [index, 종류별, 상황별, 재료별, 방법별, 제목, url, 썸네일 URL, 인분, 조리시간, 난이도, 재료, 조리순서]
Index: []


In [3]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def food_info():
    '''
    모든 요리 이름에 대한 정보를 크롤링합니다.
    '''
    base_url = "https://www.10000recipe.com"
    search_url = f"{base_url}/recipe/list.html"

    page = 1887

    while True:
        # 각 페이지를 순차적으로 크롤링
        page_url = f"{search_url}?order=reco&page={page}"
        print(f"크롤링 중: {page_url}")
        response = requests.get(page_url, verify=False)

        if response.status_code != 200:
            print(f"HTTP 응답 오류: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')

        # 레시피 링크 추출
        recipe_links = [link.get('href') for link in soup.find_all('a', class_='common_sp_link')]
        if not recipe_links:
            print(f"페이지 {page}에서 레시피를 찾을 수 없습니다.")
            break

        print(f"페이지 {page}에서 {len(recipe_links)}개의 레시피 링크를 찾았습니다.")

        page_recipes = []

        # 각 레시피 링크에 대해 크롤링을 실행
        for recipe_url in recipe_links:
            full_url = base_url + recipe_url
            recipe_info = get_recipe_info(full_url)
            if recipe_info:
                page_recipes.append(recipe_info)

        # 페이지 크롤링이 끝난 후, 모든 레시피를 DB에 저장
        save_page_to_db(page_recipes)

        # 다음 페이지로 이동
        page += 1
        time.sleep(2)  # 과도한 부하 방지를 위해 대기

def get_recipe_info(url):
    '''
    주어진 URL에서 레시피 정보를 가져옵니다.
    '''
    print(f"레시피 크롤링 중: {url}")
    response = requests.get(url)
    if response.status_code != 200:
        print(f"레시피를 가져오는 데 실패했습니다: {url}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # 요리 이름 추출
    recipe_name_element = soup.find('div', class_='view2_summary')
    recipe_name = recipe_name_element.find('h3').get_text(strip=True) if recipe_name_element else "이름 정보를 찾을 수 없음"

    # 요리 소개 추출
    intro_element = soup.find('div', class_='view2_summary_in')
    intro = intro_element.get_text(strip=True) if intro_element else "소개 정보를 찾을 수 없음"

    # 요리 정보 추출
    info_element = soup.find('div', class_='view2_summary_info')
    info = info_element.get_text(strip=True) if info_element else "정보를 찾을 수 없음"

    # 요리 사진 추출
    photo_element = soup.find('div', class_='centeredcrop')
    photo = photo_element.find('img')['src'] if photo_element else "사진 URL을 찾을 수 없음"

    # 요리 재료 추출
    ingredients_element = soup.find('div', class_='ready_ingre3')
    ingredients = ingredients_element.get_text(separator=', ', strip=True) if ingredients_element else "재료 정보를 찾을 수 없음"

    # 조리 방법 추출
    steps_element = soup.find_all('div', class_='view_step_cont')
    steps = [step.get_text(strip=True) for step in steps_element] if steps_element else ["레시피 정보를 찾을 수 없음"]

    return {
        'name': recipe_name,
        'intro': intro,
        'info': info,
        'photo': photo,
        'ingredients': ingredients,
        'recipe': steps
    }

def save_page_to_db(recipes):
    '''
    페이지의 모든 레시피 정보를 SQLite 데이터베이스에 저장합니다.
    '''
    if not recipes:
        print("저장할 레시피가 없습니다.")
        return

    conn = sqlite3.connect("recipes.db")
    cursor = conn.cursor()

    # 테이블 생성 (존재하지 않을 경우)
    cursor.execute(''' 
        CREATE TABLE IF NOT EXISTS recipes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE,
            intro TEXT,
            info TEXT,
            photo TEXT,
            ingredients TEXT NOT NULL,
            recipe TEXT NOT NULL
        )
    ''')

    for info in recipes:
        # 레시피 이름이 이미 존재하는지 확인
        cursor.execute("SELECT COUNT(*) FROM recipes WHERE name = ?", (info['name'],))
        if cursor.fetchone()[0] > 0:
            print(f"'{info['name']}' 레시피는 이미 저장되어 있습니다. 저장하지 않습니다.")
            continue

        # 데이터 삽입
        cursor.execute(''' 
            INSERT INTO recipes (name, intro, info, photo, ingredients, recipe)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (info['name'], info['intro'], info['info'], info['photo'], info['ingredients'], "\n".join(info['recipe'])))
        print(f"레시피 '{info['name']}'가 데이터베이스에 저장되었습니다.")

    conn.commit()
    conn.close()

# 프로그램 실행
if __name__ == "__main__":
    food_info()

크롤링 중: https://www.10000recipe.com/recipe/list.html?order=reco&page=1887
페이지 1887에서 40개의 레시피 링크를 찾았습니다.
레시피 크롤링 중: https://www.10000recipe.com/recipe/6934925
레시피 크롤링 중: https://www.10000recipe.com/recipe/6934943
레시피 크롤링 중: https://www.10000recipe.com/recipe/6934951
레시피 크롤링 중: https://www.10000recipe.com/recipe/6934970
레시피 크롤링 중: https://www.10000recipe.com/recipe/6934976
레시피 크롤링 중: https://www.10000recipe.com/recipe/6934980
레시피 크롤링 중: https://www.10000recipe.com/recipe/6934988
레시피 크롤링 중: https://www.10000recipe.com/recipe/6934990
레시피 크롤링 중: https://www.10000recipe.com/recipe/6935007
레시피 크롤링 중: https://www.10000recipe.com/recipe/6935008
레시피 크롤링 중: https://www.10000recipe.com/recipe/6935023
레시피 크롤링 중: https://www.10000recipe.com/recipe/6935038
레시피 크롤링 중: https://www.10000recipe.com/recipe/6935039
레시피 크롤링 중: https://www.10000recipe.com/recipe/6935049
레시피 크롤링 중: https://www.10000recipe.com/recipe/6935051
레시피 크롤링 중: https://www.10000recipe.com/recipe/6935065
레시피 크롤링 중: https://www.10000reci

KeyboardInterrupt: 